# Модель предсказания времени доставки грузов ж/д

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Чтение и первичный анализ данных

In [17]:
#Типы данных столбцов датафреймов
dislocation_dtype = {
    'AEDDTT': np.datetime64,
    'AEDTMT': np.object,
    'OBJECT_ID': np.object,
    'TU_ID': np.object,
    'ZZ_DATE_TKT': np.datetime64,
    'ZZDATE_OPER': np.datetime64,
    'ZZDATE_PROGN': np.datetime64,
    'ZZDATE_TICKET': np.datetime64,
    'ZZDISTANCE': np.float64,
    'ZZEXTTKTNR': np.object,
    'ZZKNANF': np.object,
    'ZZKNANF_NAME': np.object,
    'ZZKNEND': np.object,
    'ZZKNEND_NAME': np.object,
    'ZZST_OPER_NAME': np.object,
    'ZZSTATION_OPER': np.object
}

shipment_dtype = {
    'Завод': np.object,
    '№_Наклад': np.object,
    '№_цистер': np.object,
    'Наименование продукта': np.object,
    'ПкПогруз': np.object,
    'Бал': np.object,
    'Вес': np.float64,
    'Дата накл#': np.datetime64,
    'ФакПриб#гр': np.datetime64,
    'ПланПриб#гр': np.datetime64,
    'Вид транспортного средства': np.object,
    'Наим узла учета': np.object
}

In [14]:
dislocation = pd.read_csv(r'Дислокация.csv', delimiter=None, sep=None, dtype=dislocation_dtype)
dislocation.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,AEDDTT,OBJECT_ID,TU_ID,AEDTMT,ZZEXTTKTNR,ZZDATE_TICKET,ZZKNANF,ZZKNEND,ZZSTATION_OPER,ZZKNANF_NAME,ZZKNEND_NAME,ZZST_OPER_NAME,ZZDATE_OPER,ZZDATE_PROGN,ZZDISTANCE,ZZ_DATE_TKT
0,2017-09-01,50004357,50004357,12:01:24,ЭЙ074481,2017-08-27,/83150,/91000,/90250,Комбинатская,Тында,Небель (рзд),2017-09-01,NaT,1497.0,2017-08-27
1,2017-09-01,50004456,50004456,12:01:24,ЭЙ244245,2017-08-31,/83150,/85010,/85000,Комбинатская,Обь,Инская,2017-09-01,NaT,27.0,2017-08-31
2,2017-09-01,50004654,50004654,12:01:24,ЭЙ155499,2017-08-29,/83150,/79800,/79660,Комбинатская,Нижневартовск 1,Демьянка,2017-09-01,NaT,511.0,2017-08-29
3,2017-09-01,50004787,50004787,12:01:24,23809535,2017-08-13,/83150,/93710,/93690,Комбинатская,Наушки (эксп. Мтз:Улан-Ба,Наушки,2017-08-23,NaT,6.0,2017-08-13
4,2017-09-01,50004902,50004902,12:01:24,ЭЙ290728,2017-09-01,/83150,/90440,/83000,Комбинатская,Таксимо,Московка,2017-09-01,NaT,3268.0,2017-08-31


In [15]:
dislocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494638 entries, 0 to 494637
Data columns (total 16 columns):
AEDDTT            494638 non-null datetime64[ns]
OBJECT_ID         494638 non-null object
TU_ID             494638 non-null object
AEDTMT            494638 non-null object
ZZEXTTKTNR        486432 non-null object
ZZDATE_TICKET     391488 non-null datetime64[ns]
ZZKNANF           494638 non-null object
ZZKNEND           494638 non-null object
ZZSTATION_OPER    494638 non-null object
ZZKNANF_NAME      494563 non-null object
ZZKNEND_NAME      494638 non-null object
ZZST_OPER_NAME    491442 non-null object
ZZDATE_OPER       494638 non-null datetime64[ns]
ZZDATE_PROGN      111865 non-null datetime64[ns]
ZZDISTANCE        494638 non-null float64
ZZ_DATE_TKT       470613 non-null datetime64[ns]
dtypes: datetime64[ns](5), float64(1), object(10)
memory usage: 60.4+ MB


In [4]:
dislocation.loc[~pd.isnull(dislocation['ZZEXTTKTNR'])]

,AEDDTT,OBJECT_ID,TU_ID,AEDTMT,ZZEXTTKTNR,ZZDATE_TICKET,ZZKNANF,ZZKNEND,ZZSTATION_OPER,ZZKNANF_NAME,ZZKNEND_NAME,ZZST_OPER_NAME,ZZDATE_OPER,ZZDATE_PROGN,ZZDISTANCE,ZZ_DATE_TKT
0,2017-09-01 00:00:00.000,50004357,50004357,12:01:24,ЭЙ074481,2017-08-27 00:00:00.000,/83150,/91000,/90250,Комбинатская,Тында,Небель (рзд),2017-09-01 00:00:00.000,NaN,1497,2017-08-27 00:00:00.000
1,2017-09-01 00:00:00.000,50004456,50004456,12:01:24,ЭЙ244245,2017-08-31 00:00:00.000,/83150,/85010,/85000,Комбинатская,Обь,Инская,2017-09-01 00:00:00.000,NaN,27,2017-08-31 00:00:00.000
2,2017-09-01 00:00:00.000,50004654,50004654,12:01:24,ЭЙ155499,2017-08-29 00:00:00.000,/83150,/79800,/79660,Комбинатская,Нижневартовск 1,Демьянка,2017-09-01 00:00:00.000,NaN,511,2017-08-29 00:00:00.000
3,2017-09-01 00:00:00.000,50004787,50004787,12:01:24,23809535,2017-08-13 00:00:00.000,/83150,/93710,/93690,Комбинатская,Наушки (эксп. Мтз:Улан-Ба,Наушки,2017-08-23 00:00:00.000,NaN,6,2017-08-13 00:00:00.000
4,2017-09-01 00:00:00.000,50004902,50004902,12:01:24,ЭЙ290728,2017-09-01 00:00:00.000,/83150,/90440,/83000,Комбинатская,Таксимо,Московка,2017-09-01 00:00:00.000,NaN,3268,2017-08-31 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494633,2017-11-12 00:00:00.000,51621712,51621712,12:01:18,ЭМ429354,2017-11-07 00:00:00.000,/83150,/52270,/61100,Комбинатская,Юровский,Им.максима горького,2017-11-12 00:00:00.000,NaN,831,2017-11-08 00:00:00.000
494634,2017-11-12 00:00:00.000,51621753,51621753,12:01:18,ЭМ590454,2017-11-10 00:00:00.000,/83150,/03580,/76000,Комбинатская,Новый порт,Пермь-сортировочная,2017-11-12 00:00:00.000,NaN,1714,2017-11-10 00:00:00.000
494635,2017-11-12 00:00:00.000,51621787,51621787,12:01:18,ЭМ602375,2017-11-10 00:00:00.000,/80050,/80310,/80000,Синеглазово,Аргаяш,Челябинск-главный,2017-11-11 00:00:00.000,NaN,56,2017-11-10 00:00:00.000
494636,2017-11-12 00:00:00.000,51621860,51621860,12:01:18,ЭМ388623,2017-11-06 00:00:00.000,/83150,/52090,/51840,Комбинатская,Новороссийск,Сальск,2017-11-12 00:00:00.000,NaN,418,2017-11-06 00:00:00.000


In [18]:
shipment = pd.read_csv(r'Факт отгрузки.csv', delimiter=None, sep=None, dtype=shipment_dtype)
shipment.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,№_Наклад,№_цистер,Завод,Наименование продукта,ПкПогруз,Бал,Вес,Дата накл#,ФакПриб#гр,ПланПриб#гр,Вид транспортного средства,Наим узла учета
0,23745911,50384577,MNPZ,"СМТ (DMA),вид Э",ст. Яничкино,25D,1.504110e+27,2017-08-01,2017-08-01,2017-08-09,Железнодорожный,МНПЗ-ж/д
1,23745911,50637008,MNPZ,"СМТ (DMA),вид Э",ст. Яничкино,25D,5.321670e+26,2017-08-01,2017-08-01,2017-08-09,Железнодорожный,МНПЗ-ж/д
2,23745911,50644855,MNPZ,"СМТ (DMA),вид Э",ст. Яничкино,25D,7.915220e+26,2017-08-01,2017-08-01,2017-08-09,Железнодорожный,МНПЗ-ж/д
3,23745911,50656842,MNPZ,"СМТ (DMA),вид Э",ст. Яничкино,25D,1.496610e+27,2017-08-01,2017-08-01,2017-08-09,Железнодорожный,МНПЗ-ж/д
4,23745911,50667427,MNPZ,"СМТ (DMA),вид Э",ст. Яничкино,25D,1.999280e+26,2017-08-01,2017-08-01,2017-08-09,Железнодорожный,МНПЗ-ж/д


In [19]:
shipment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121371 entries, 0 to 121370
Data columns (total 12 columns):
№_Наклад                      121371 non-null object
№_цистер                      121371 non-null object
Завод                         121371 non-null object
Наименование продукта         121371 non-null object
ПкПогруз                      121371 non-null object
Бал                           121371 non-null object
Вес                           121371 non-null float64
Дата накл#                    121371 non-null datetime64[ns]
ФакПриб#гр                    121371 non-null datetime64[ns]
ПланПриб#гр                   120248 non-null datetime64[ns]
Вид транспортного средства    121371 non-null object
Наим узла учета               121371 non-null object
dtypes: datetime64[ns](3), float64(1), object(8)
memory usage: 11.1+ MB
